In [1]:
import robotic as ry
import numpy as np
import time

In [2]:
config = ry.Config()
config.addFile("puzzles/p9-cube-free.g")

In [3]:
print(config.getFrameNames())

['world', 'floor', 'outwall_right', 'outwall_back', 'outwall_left', 'outwall_front', 'sub-goal1', 'world', 'floor', 'outwall_right', 'outwall_back', 'outwall_left', 'outwall_front', 'sub-goal1', 'camera_top', 'egoJoint', 'ego', 'goal', 'goal_visible', 'obj', 'goalLarge']


In [4]:
EGO_NAME = "ego"
OBJ_NAME = "obj"
SUB_GOAL_NAME = "sub-goal1"
GOAL_NAME = "goal"
CAMERA_NAME = "camera_top"
FLOOR_NAME = "floor"

GAP = config.getFrame(EGO_NAME).getSize()[1] * 3

In [5]:
config.view() # somehow it breaks if I don't call this before setting the camera
f = config.getFrame('camera_top')
config.view_setCamera(f)

In [6]:
ry.params_clear()
ry.params_add({
    "rrt/stepsize": 0.05,
    "rrt/verbose": 2
})

In [7]:
q_start = config.getFrame(EGO_NAME).getPosition()[:2]
q_obj = config.getFrame(OBJ_NAME).getPosition()[:2]
q_goal = config.getFrame(GOAL_NAME).getPosition()[:2]

In [8]:
def cartesian_to_polar(coordinates):
    x = coordinates[0]
    y = coordinates[1]
    r = np.hypot(x, y)
    theta = np.arctan2(y, x)
    return r, theta

In [9]:
def polar_to_cartesian(coordinates):
    r = coordinates[0]
    theta = coordinates[1]
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return x, y

In [10]:
def solve_touch_with_komo():
    komo = ry.KOMO(config, phases=1, slicesPerPhase=1, kOrder=0, enableCollisions=False)
    komo.addObjective([], ry.FS.negDistance, [EGO_NAME, OBJ_NAME], ry.OT.eq, [1e1])
    ret = ry.NLP_Solver(komo.nlp(), verbose=3).solve()
    if ret:
        path = komo.getPath()
        return path
    else:
        return None

In [11]:
def solve_ik_with_komo(q_home, q_target):
    komo = ry.KOMO(config, phases=1, slicesPerPhase=1, kOrder=0, enableCollisions=False)
    komo.addObjective([], ry.FS.position, [EGO_NAME, OBJ_NAME], ry.OT.eq, [1e1])
    ret = ry.NLP_Solver(komo.nlp(), verbose=3).solve()
    if ret:
        path = komo.getPath()
        return path
    else:
        return None

In [11]:
def solve_rrt(q_start, q_obj, q_goal):
    ITERATIONS = 24
    delta_theta = 2 * np.pi / ITERATIONS
    for i in range(ITERATIONS):
        transpose_polar1 = np.array([GAP, i * delta_theta])
        transpose_cartesian1 = np.array(polar_to_cartesian(transpose_polar1))
        q_home1 = q_obj + transpose_cartesian1
        config.setJointState(q_home1)
        q_subgoal = solve_touch_with_komo()
        if q_subgoal is not None:
            config.setJointState(q_start)
            rrt = ry.PathFinder()
            rrt.setProblem(config, [q_start], [q_subgoal])
            solution1 = rrt.solve()
            del rrt
            
            config.setJointState(q_subgoal)
            config.attach(EGO_NAME, OBJ_NAME)
            q_real_goal = q_goal + q_subgoal - q_obj

            rrt = ry.PathFinder()
            rrt.setProblem(config, [q_subgoal], [q_real_goal])
            solution2 = rrt.solve()
            config.getFrame(OBJ_NAME).unLink()
            config.setJointState(q_start)
            config.view()
            if solution1.feasible and solution2.feasible:
                return q_subgoal, solution1, solution2

    config.setJointState(q_start)            
    return None, None, None

In [12]:
# RRT from start to object
q_subgoal, solution1, solution2 = solve_rrt(q_start, q_obj, q_goal)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1 	lambda:[]
----newton---- initial point f(x):6.37627 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):   0.275759  ACCEPT
--newton-- it:   2  |Delta|:  0.0522515  alpha:          1  evals:   3  f(y):6.82555e-06  ACCEPT
--newton-- it:   3  |Delta|:0.000259958 	 -- absMax(Delta)<1e-1*o.stopTolerance -- NO UPDATE
==nlp== it:   0  evals:   3  A(x):6.82555e-06  f:          0  g:          0  h: 0.00261257  |x-x'|:   0.252251 	stop:DeltaConverge 	x:[1.25026, -1.50968]
==nlp== it:   1  evals:   3  A(x):4.77788e-05  mu:5 	lambda:[-0.00522515]
--newton-- it:   4  |Delta|:0.000313196 	 -- absMax(Delta)<1e-1*o.stopTolerance -- NO UPDATE
==nlp== it:   1  evals:   3  A(x):4.77788e-05  f:          0  g:          0  h: 0.00261257  |x-x'|:          0 	stop:DeltaConverge 	x:[1.25026, -1.50968]
==nlp== StoppingCriterion Delta<0.01
----newton---- final f(x):4.77788e-05
  -

In [13]:
for state in solution1.x:
    prev_state = state

    config.setJointState(state)
    config.view()
    time.sleep(3./solution1.x.shape[0]) # animate in 3 seconds

In [14]:
config.attach(EGO_NAME, OBJ_NAME)

# TODO try to use KOMO touch - stable constraints
# touch -> negDistance
# stable -> self.komo.addModeSwitch([1.,-1.], ry.SY.stable, [gripper, obj], True)
# 
# KOMO/verbose ile oyna
# komo.report()

In [15]:
for state in solution2.x:
    prev_state = state

    config.setJointState(state)
    config.view()
    time.sleep(3./solution2.x.shape[0]) # animate in 3 seconds

In [16]:
config.view_close()
del config